In [1]:
#Read tif volume
import numpy as np
import tifffile as tiff
import os
import sys
import matplotlib.pyplot as plt

def read_tif_volume(tif_file):
    #Read tif volume
    volume = tiff.imread(tif_file)
    return volume

def write_tif_volume(volume, tif_file):
    tiff.imwrite(tif_file, volume)
    
skeleton = read_tif_volume('outputs/output_skeleton_volume.tif')

print(f"Number of pixels in the volume: {np.size(skeleton)}")
print(f"Number of pixels set to True: {np.sum(skeleton==True)}")
print(f"Number of pixels set to False: {np.sum(skeleton == False, dtype=np.int64)}")

sk_copy = skeleton.copy()


Number of pixels in the volume: 2666222100
Number of pixels set to True: 133044
Number of pixels set to False: 2666089056


In [3]:
import numpy as np
from skan import Skeleton, summarize

# Skeleton volume is already loaded as 'sk_copy'
skel = Skeleton(sk_copy)

# Summarize the skeleton branch information
branch_data = summarize(skel)


# Identify branches of different types
branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

# Print branch counts
print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")

# Let's say we want to delete branches of type 0 and type 1
branches_to_delete = branch_type_0.index.tolist() + branch_type_3.index.tolist()

# Loop through the branch indices to remove the entire branch
for branch_id in branches_to_delete:
    # Retrieve all pixel coordinates along the branch
    coordinates = skel.path_coordinates(branch_id)
    
    # Set corresponding pixels in the skeleton copy to False (or 0) to remove the branch
    for coord in coordinates:
        sk_copy[tuple(np.round(coord).astype(int))] = False  # Ensure coordinates are integers

# Count the number of branches of each type after deletion
skel = Skeleton(sk_copy)
branch_data = summarize(skel)
branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")


# Save the modified skeleton to a new file
# write_tif_volume(sk_copy, 'outputs/modified_skeleton_volume.tif')

# Optional: Visualize or analyze the new skeleton if needed


/tmp/ipykernel_971658/4215341416.py:8: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


Number of branch type 0: 937
Number of branch type 1: 1812
Number of branch type 2: 1640
Number of branch type 3: 47
Number of branch type 0: 14
Number of branch type 1: 1815
Number of branch type 2: 1590
Number of branch type 3: 4


/tmp/ipykernel_971658/4215341416.py:37: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [3]:
write_tif_volume(sk_copy, 'outputs/modified_skeleton_volume.tif')